In [0]:
# 匯入「keras_lite_convertor.py」到 Colab

from google.colab import files 
uploaded = files.upload()       # 匯入檔案

Saving keras_lite_convertor.py to keras_lite_convertor.py


In [0]:
# 讀取 walk_data 資料夾內的檔案

import keras_lite_convertor as kc

path_name = 'walk_data'   
Data_reader = kc.Data_reader(path_name, 
                             mode='binary',    # binary 適用於二元分類
                             label_name = ['others','walk'])  # 標籤名稱
data, label = Data_reader.read()

In [0]:
# 標籤   1代表 walk、0代表 others

label

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1])

In [0]:
# 資料預處理

# 取資料中的 90% 當作訓練集
split_num = int(len(data)*0.9) 
train_data=data[:split_num]
train_label=label[:split_num]    

# 正規化
mean = train_data.mean() # 平均數
data -= mean
std = train_data.std()   # 標準差
data /= std

In [0]:
# 訓練集、驗證集、測試集的資料形狀

# 訓練集
print(train_data.shape)    

# 驗證集
validation_data=data[split_num:-5]  
print(validation_data.shape)
validation_label=label[split_num:-5]   

# 測試集
test_data=data[-5:]     
print(test_data.shape)
test_label=label[-5:]  

(180, 102)
(15, 102)
(5, 102)


In [0]:
# 建立神經網路架構

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# 建立神經網路物件
model = Sequential()

# 增加輸入層
model.add(layers.Dense(10,activation = 'relu',
                       input_shape=(102,)))
# 輸出層的啟動函數為 sigmoid(輸出值介於 0~1 之間, 適合用於二元分類的神經網路) 
model.add(layers.Dense(1,activation = 'sigmoid')) 

# 顯示模型資訊
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                1030      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________


In [0]:
# 編譯及訓練模型

model.compile(optimizer='adam',loss='binary_crossentropy',
              metrics=['acc'])
train_history = model.fit(train_data,train_label,   
      validation_data=(validation_data,validation_label),  
      epochs=300)   

Epoch 1/300
6/6 [==============================] - 0s 44ms/step - loss: 0.6488 - acc: 0.5444 - val_loss: 0.5381 - val_acc: 0.7333
Epoch 2/300
6/6 [==============================] - 0s 6ms/step - loss: 0.5885 - acc: 0.6278 - val_loss: 0.4834 - val_acc: 0.7333
Epoch 3/300
6/6 [==============================] - 0s 6ms/step - loss: 0.5436 - acc: 0.6556 - val_loss: 0.4465 - val_acc: 0.7333
Epoch 4/300
6/6 [==============================] - 0s 6ms/step - loss: 0.5035 - acc: 0.7222 - val_loss: 0.4133 - val_acc: 0.8000
Epoch 5/300
6/6 [==============================] - 0s 6ms/step - loss: 0.4739 - acc: 0.7500 - val_loss: 0.3814 - val_acc: 0.8667
Epoch 6/300
6/6 [==============================] - 0s 6ms/step - loss: 0.4451 - acc: 0.7833 - val_loss: 0.3605 - val_acc: 0.9333
Epoch 7/300
6/6 [==============================] - 0s 6ms/step - loss: 0.4207 - acc: 0.8000 - val_loss: 0.3416 - val_acc: 0.9333
Epoch 8/300
6/6 [==============================] - 0s 6ms/step - loss: 0.3970 - acc: 0.8500 - va

In [0]:
# 測試模型

print('predict:')
print(model.predict(test_data))
print()
print('real:')
print(test_label)

predict:
[[9.9988079e-01]
 [2.5223169e-04]
 [9.9951291e-01]
 [9.9997926e-01]
 [9.9995196e-01]]

real:
[1 0 1 1 1]


In [0]:
# 儲存模型

kc.save(model,'walk_model.json')

In [0]:
# 顯示資訊

print('mean=', mean)
print('std=', std)

mean= 849.1830065359477
std= 16766.660464036682
